<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT22/blob/filippaolofsson-lab3/Lab3/filippaolofsson_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab: iterative methods**
**Filippa Olofsson**

# **Abstract**

This lab consists of three assignments, Jacobi iteration and Gauss-Seidel iteration for Ax=b, and Newton's method for scalar nonlinear equation f(x)=0. 

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [2]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

In this lab three different iterative methods for solving equations is made. The first two are for solving matrix equations on the form Ax=b and the third one is the Newtons method for approximating x in a solution of a nonlinear equation f(x) = 0. 


# **Method**

### **1. Function: Jacobi iteration for Ax=b**

The Jacobi iteration is a stationary iterative method to find a solution of x for equations Ax=b. Jacobo iteration is shown to converge if A is diagonally dominant(Chapter 7, p. 152). Therefore the algorithm checks if the matrix A is diagonally dominant, with the equation: 
$|a_{ii}| \geq \sum_{i\neq j} |a_{ij}| $ 

The convergence criterion $\parallel$M$\parallel$ < 1 is also checked if the matrix A is not diagonally dominant. 

The function Jacobi iteration is based on the formulas in example 7.8, Chapter 7, which iterativly finds a solution x.

$x_{i}^{(k+1)} = a_{ii}^{-1} \left( b_i - \sum_{j\neq i} a_{ij}x_j^{(k)} \right)$

The initial vector x is a zero-vector.  

In [3]:
def diagonally_dominant(D, R, n):
    for i in range(0, n):
      D_sum = np.sum(np.abs(D[i]))
      R_sum = np.sum(np.abs(R[i]))
      if D_sum < R_sum or (D_sum == 0 and R_sum == 0):
        return False
    return True

In [4]:
#Input: matrix A, vector b
#Output: vector x

def Jacobi_iteration(A, b):
  n = len(b)
  D = np.diag(np.diag(A))
  R = A - D
  Dinv = np.linalg.inv(D)
  M_norm = np.linalg.norm(np.identity(n) - np.dot(Dinv, A))
  dia_dom = diagonally_dominant(D, R, n)
  if dia_dom is False and M_norm >= 1:
    return "Will not converge"
  
  x1 = np.zeros(n)
  x2 = np.zeros(n)
  residual = np.linalg.norm(np.dot(A, x2) - b)
  while residual > 1e-10:
    for i in range(0, n):
      sum = 0
      for j in range(0, n):
        if j != i:
          sum += A[i][j]*x1[j]
      x2[i] = Dinv[i][i] * (b[i] - sum)
    for k in range(0, n):
      x1[k] = x2[k]
    residual = np.linalg.norm(np.dot(A, x2) - b)
  return x2


### **2. Function: Gauss-Seidel iteration for Ax=b**

The Gauss-Seidel iteration function is based on the Example 7.9. It is an iterative method to finding a solution of x for equations Ax = b. The function checks if the matrix A is diagonally dominant and if any diagonal element in A = 0. The convergence criterion $\parallel Ι - L^{-1}$A $\parallel < 1$, is also considered when the matrix A is not diagonally dominant. The initial vector x is a zero-vector.

$x_{i}^{(k+1)} = a_{ii}^{-1} \left( b_i - \sum_{j < i} a_{ij}x_j^{(k+1)} - \sum_{j > i} a_{ij}x_j^{(k)}\right)$

In [5]:
#nput: matrix A, vector b
#Output: vector x

def Gauss_Seidel_iteration(A, b):
  n = len(b) 
  D = np.diag(np.diag(A))
  R = A - D
  L = np.tril(A)
  M = np.linalg.norm(np.identity(n) - np.dot(np.linalg.inv(L), A))
  dia_dom = diagonally_dominant(D, R, n)
  if dia_dom == False and M > 1:
    return "Will not converge"
  
  x1 = np.zeros(n)
  x2 = np.zeros(n)
  residual = np.linalg.norm(np.dot(A, x2) - b)
  while residual > 1e-10:
    for i in range(0, n):
      sum1 = 0
      sum2 = 0
      for j in range(0, n):
        if j < i:
          sum1 += A[i][j]*x2[j]
        elif j > i:
          sum2 += A[i][j]*x1[j]
      x2[i] = (1/A[i][i]) * (b[i] - sum1 - sum2)
      for k in range(0, n):
        x1[i] = x2[i]
      residual = np.linalg.norm(np.dot(A, x2) - b)
  return x2 

### **3. Function: Newton's method for scalar nonlinear equation f(x)=0**

The function Newtons method is used to find an approximate solution to a nonlinear function f(x) = 0. 
This algorithm is based on algorithm 8.2, Chapter 8. The function begins with an initial guess of x and then iteratively computing a new x until the function f(x) is close to zero. 

$ x^{(k+1)} = x^{(k)} - \frac{f(x^{(k)})}{f'(x^{(k)})}$

In [6]:
#Input: scalar function f(x)
#Output: real number x

def Newtons_method(f, x0):
  x = x0 #initial guess 
  TOL = 1e-10
  h = 1e-10
  while np.abs(f(x)) > TOL:
    df = (f(x + h) - f(x)) / h
    x = x - f(x)/df
  return x

# **Results**

There are 3 testcases for each problem, the tests are described with comments in the code below. 

In [7]:
A1 = np.array([[9, 7], 
               [6, 6]])
A2 = np.array([[10, 5],
               [0, 6]])
A3 = np.array([[8, 3, 2], 
               [4, 11, 0], 
               [6, 2, 9]])

b1 = np.array([6, 7])
b2 = np.array([1, 0])
b3 = np.array([10, 1, 7])

# 1. Tests Jacobi iteration
x1 = Jacobi_iteration(A1, b1)
x2 = Jacobi_iteration(A2, b2)
x3 = Jacobi_iteration(A3, b3)

# || x-y || for manufactured/exact solution y 
assert np.linalg.norm(x1 - np.dot(np.linalg.inv(A1), b1)) <= 1e-10
assert np.linalg.norm(x2 - np.dot(np.linalg.inv(A2), b2)) <= 1e-10
assert np.linalg.norm(x3 - np.dot(np.linalg.inv(A3), b3)) <= 1e-10

# || Ax-b || 
assert np.linalg.norm(np.dot(A1, x1) - b1) <= 1e-10
assert np.linalg.norm(np.dot(A2, x2) - b2) <= 1e-10
assert np.linalg.norm(np.dot(A3, x3) - b3) <= 1e-10

print("Test 1: passed")


# 2. Tests Gauss Seidel iteration
y1 = Gauss_Seidel_iteration(A1, b1)
y2 = Gauss_Seidel_iteration(A2, b2)
y3 = Gauss_Seidel_iteration(A3, b3)

# || x-y || for manufactured/exact solution y
assert np.linalg.norm(x1 - np.dot(np.linalg.inv(A1), b1)) <= 1e-10
assert np.linalg.norm(x2 - np.dot(np.linalg.inv(A2), b2)) <= 1e-10
assert np.linalg.norm(x3 - np.dot(np.linalg.inv(A3), b3)) <= 1e-10

# || Ax-b || 
assert np.linalg.norm(np.dot(A1, x1) - b1) <= 1e-10
assert np.linalg.norm(np.dot(A2, x2) - b2) <= 1e-10
assert np.linalg.norm(np.dot(A3, x3) - b3) <= 1e-10

print("Test 2: passed")


# 3. Tests Newtons Method
def f1(x):
  return np.cos(x) 

def f2(x):
  return np.sin(x) + x

def f3(x):
  return x**2 - 3

x1 = Newtons_method(f1, 1)
x2 = Newtons_method(f2, 1)
x3 = Newtons_method(f3, 1)

# convergence of residual |f(x)|
assert np.linalg.norm(f1(x1)) <= 1e-10
assert np.linalg.norm(f2(x2)) <= 1e-10
assert np.linalg.norm(f3(x3)) <= 1e-10

# |x-y| for manufactured/exact solution y
assert (x1 - np.pi/2) <= 1e-10
assert (x2 - 0) <= 1e-10
assert (x3 - np.sqrt(3)) <= 1e-10

print("Test 3: passed")

Test 1: passed
Test 2: passed
Test 3: passed


# **Discussion**

I think this lab was interesting and the results where expected. 